<a href="https://colab.research.google.com/github/Nisha-tk/NLP-project/blob/main/resume_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd



In [3]:
Data= pd.read_csv("/content/jobs_dataset_with_features.csv")

In [4]:
Data.head(2)

,Role,Features
0,Social Media Manager,5 to 15 Years Digital Marketing Specialist M.T...
1,Frontend Web Developer,"2 to 12 Years Web Developer BCA HTML, CSS, Jav..."


In [5]:
Data["Role"].value_counts()

,count
Role,
Interaction Designer,1031
Network Administrator,850
User Interface Designer,780
Social Media Manager,701
User Experience Designer,685
...,...
Legal Counsel,145
Small Animal Veterinarian,145
Crisis Communication Manager,145


In [7]:
Data.shape

(82323, 2)

In [8]:
min_count = 200  # Or another value after testing

role_counts = Data['Role'].value_counts()
dropped_classes = role_counts[role_counts < min_count].index
filtered_df = Data[~Data['Role'].isin(dropped_classes)].reset_index(drop=True)

# Check the updated role counts to verify the change
filtered_df['Role'].value_counts()

,count
Role,
Interaction Designer,1031
Network Administrator,850
User Interface Designer,780
Social Media Manager,701
User Experience Designer,685
...,...
Deliverability Analyst,206
JavaScript Developer,204
Advertising Account Executive,204


In [9]:
filtered_df.shape


(28619, 2)

In [10]:
df = filtered_df.sample(n=10000)

In [11]:
df.shape

(10000, 2)

In [13]:
df.head()

,Role,Features
25265,Customer Success Manager,2 to 13 Years Customer Support Specialist BA C...
3294,Inside Sales Representative,4 to 12 Years Sales Representative BCA Sales p...
26260,Procurement Manager,5 to 12 Years Procurement Specialist MCA Procu...
9123,Office Manager,0 to 9 Years Executive Assistant MCA Office ad...
2985,Copywriter,4 to 9 Years Content Writer B.Tech Copywriting...


,Role,Features
25265,Customer Success Manager,2 to 13 Years Customer Support Specialist BA C...
3294,Inside Sales Representative,4 to 12 Years Sales Representative BCA Sales p...
26260,Procurement Manager,5 to 12 Years Procurement Specialist MCA Procu...
9123,Office Manager,0 to 9 Years Executive Assistant MCA Office ad...
2985,Copywriter,4 to 9 Years Content Writer B.Tech Copywriting...


# TFIDf vectorization

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report


In [19]:
x=df["Features"]
y=df["Role"]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

tfidf_vectorizer=TfidfVectorizer()
x_train_tfidf=tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf=tfidf_vectorizer.transform(x_test)



In [20]:
print(f"x_train shape: {x_train_tfidf.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test_tfidf.shape}")
print(f"y_test shape: {y_test.shape}")


x_train shape: (8000, 8628)
y_train shape: (8000,)
x_test shape: (2000, 8628)
y_test shape: (2000,)


In [21]:
rf_classifier=RandomForestClassifier(n_estimators=100,random_state=42)
rf_classifier.fit(x_train_tfidf,y_train)
y_pred=rf_classifier.predict(x_test_tfidf)
accuracy=accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [22]:
import re
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText


# Prediction and Category Name
def job_recommendation(resume_text):
    resume_text= cleanResume(resume_text)
    resume_tfidf = tfidf_vectorizer.transform([resume_text])
    predicted_category = rf_classifier.predict(resume_tfidf)[0]
    return predicted_category

In [23]:
resume_file = """Objective:
A creative and detail-oriented Designer with a passion for visual communication and brand identity seeking opportunities to leverage design skills in a dynamic and collaborative environment.

Education:
- Bachelor of Fine Arts in Graphic Design, XYZ College, GPA: 3.7/4.0
- Diploma in Web Design, ABC Institute, GPA: 3.9/4.0

Skills:
- Proficient in Adobe Creative Suite (Photoshop, Illustrator, InDesign)
- Strong understanding of typography, layout, and color theory
- Experience in both print and digital design
- Ability to conceptualize and execute design projects from concept to completion
- Excellent attention to detail and time management skills

Experience:
Graphic Designer | XYZ Design Studio
- Created visually appealing graphics for various marketing materials, including brochures, flyers, and social media posts
- Collaborated with clients to understand their design needs and deliver creative solutions that align with their brand identity
- Worked closely with the marketing team to ensure consistency in brand messaging across all platforms

Freelance Designer
- Designed logos, branding materials, and website layouts for small businesses and startups
- Managed multiple projects simultaneously while meeting tight deadlines and maintaining quality standards
- Established and maintained strong client relationships through clear communication and exceptional service

Projects:
- Rebranding Campaign for XYZ Company: Led a team to redesign the company's logo, website, and marketing collateral, resulting in a 30% increase in brand recognition
- Packaging Design for ABC Product Launch: Developed eye-catching packaging designs for a new product line, contributing to a successful launch and positive customer feedback

Certifications:
- Adobe Certified Expert (ACE) in Adobe Illustrator
- Responsive Web Design Certification from Udemy

Languages:
- English (Native)
- Spanish (Intermediate)
"""

job_category = job_recommendation(resume_file)
print(f"Recommended Job Category: {job_category}")

Recommended Job Category: User Interface Designer


In [24]:
import pickle
pickle.dump(rf_classifier,open('rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('tfidf_vectorizer_job_recommendation.pkl','wb'))